In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import tensorflow as tf
import zipfile
from sklearn.preprocessing import RobustScaler

# Option a) local file contains a small subset of the entire dataset
url = "D:/fraunhofer_eas_dataset_for_unbalance_detection_v1.zip"

# Option b) the entire dataset can be directly downloaded via public Fraunhofer Fortadis dataspace
#url = 'https://fordatis.fraunhofer.de/bitstream/fordatis/151.2/1/fraunhofer_eas_dataset_for_unbalance_detection_v1.zip'

# Option c) selected pre-trained models can be found in the directory model/reference
use_reference_models = True
model_path = '../models/reference'

# Option d) all models will be trained again
#use_reference_models = False
#model_path = '../models'

with zipfile.ZipFile(url, 'r') as f:
    with f.open('0D.csv', 'r') as c:
        data0D = pd.read_csv(c)
        print('done')
    with f.open('0E.csv', 'r') as c:
        data0E = pd.read_csv(c)
        print('done')
    with f.open('1D.csv', 'r') as c:
        data1D = pd.read_csv(c)
        print('done')
    with f.open('1E.csv', 'r') as c:
        data1E = pd.read_csv(c)
        print('done')
    with f.open('2D.csv', 'r') as c:
        data2D = pd.read_csv(c)
        print('done')
    with f.open('2E.csv', 'r') as c:
        data2E = pd.read_csv(c)
        print('done')
    with f.open('3D.csv', 'r') as c:
        data3D = pd.read_csv(c)
        print('done')
    with f.open('3E.csv', 'r') as c:
        data3E = pd.read_csv(c)
        print('done')
    with f.open('4D.csv', 'r') as c:
        data4D = pd.read_csv(c)
        print('done')
    with f.open('4E.csv', 'r') as c:
        data4E = pd.read_csv(c)
        print('done')

        labels = {'no_unbalance':0, 'unbalance':1}
sensor = 'Vibration_1'
samples_per_second = 4096
seconds_per_analysis = 1.0
window = int(samples_per_second*seconds_per_analysis)

def get_features(data, label):
    n = int(np.floor(len(data)/window))
    data = data[:int(n)*window]
    X = data.values.reshape((n, window))
    y = np.ones(n)*labels[label]
    return X,y

X0,y0 = get_features(data0D[sensor], "no_unbalance")
X1,y1 = get_features(data1D[sensor], "unbalance")
X2,y2 = get_features(data2D[sensor], "unbalance")
X3,y3 = get_features(data3D[sensor], "unbalance")
X4,y4 = get_features(data4D[sensor], "unbalance")
X=np.concatenate([X0, X1, X2, X3, X4])
y=np.concatenate([y0, y1, y2, y3, y4])

X0_val, y0_val = get_features(data0E[sensor], "no_unbalance")
X1_val, y1_val = get_features(data1E[sensor], "unbalance")
X2_val, y2_val = get_features(data2E[sensor], "unbalance")
X3_val, y3_val = get_features(data3E[sensor], "unbalance")
X4_val, y4_val = get_features(data4E[sensor], "unbalance")
X_val=np.concatenate([X0_val, X1_val, X2_val, X3_val, X4_val])

print(X.shape, y.shape, X_val.shape, y_val.shape)

from sklearn.model_selection import train_test_split
train_test_ratio = 0.9
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 1-train_test_ratio, random_state = 0)
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

X_fft = np.abs(np.fft.rfft(X, axis=1))[:,:int(window/2)]
X_train_fft = np.abs(np.fft.rfft(X_train, axis=1))[:,:int(window/2)]
X_test_fft = np.abs(np.fft.rfft(X_test, axis=1))[:,:int(window/2)]
X_val_fft = np.abs(np.fft.rfft(X_val, axis=1))[:,:int(window/2)]

X_fft[:,0]=0
X_train_fft[:,0]=0
X_test_fft[:,0]=0
X_val_fft[:,0]=0

print(X_train_fft.shape, X_test_fft.shape, X_val_fft.shape)
scaler = RobustScaler(quantile_range=(5,95)).fit(X_train_fft)

X_fft_sc = scaler.transform(X_fft)
X_train_fft_sc = scaler.transform(X_train_fft)
X_test_fft_sc = scaler.transform(X_test_fft)
X_val_fft_sc = scaler.transform(X_val_fft)

X_val_fft_1 = X_val_fft_sc[:len(y0_val),:]
y_val_1 = y_val[:len(y0_val)]
X_val_fft_2 = X_val_fft_sc[len(y0_val):len(y0_val)+len(y1_val),:]
y_val_2 = y_val[len(y0_val):len(y0_val)+len(y1_val)]
X_val_fft_3 = X_val_fft_sc[len(y0_val)+len(y1_val):len(y0_val)+
                           len(y1_val)+len(y2_val),:]
y_val_3 = y_val[len(y0_val)+len(y1_val):len(y0_val)+len(y1_val)+
                len(y2_val)]
X_val_fft_4 = X_val_fft_sc[len(y0_val)+len(y1_val)+len(y2_val):len(y0_val)+
                           len(y1_val)+len(y2_val)+len(y3_val),:]
y_val_4 = y_val[len(y0_val)+len(y1_val)+len(y2_val):len(y0_val)+len(y1_val)+
                len(y2_val)+len(y3_val)]
X_val_fft_5 = X_val_fft_sc[len(y0_val)+len(y1_val)+len(y2_val)+len(y3_val):len(y0_val)+
                           len(y1_val)+len(y2_val)+len(y3_val)+len(y4_val),:]
y_val_5 = y_val[len(y0_val)+len(y1_val)+len(y2_val)+len(y3_val):len(y0_val)+len(y1_val)+
                len(y2_val)+len(y3_val)+len(y4_val)]

from tensorflow.keras.models import Sequential, load_model, Model
from tensorflow.keras.layers import BatchNormalization,LeakyReLU,Dense,Dropout
from tensorflow.keras.layers import Input,Conv1D,MaxPooling1D,Flatten,ReLU
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.regularizers import l1_l2

if not use_reference_models:

    weight_for_0 = len(y)/(2*len(y[y==0]))
    weight_for_1 = len(y)/(2*len(y[y==1]))
    class_weight = {0: weight_for_0, 1: weight_for_1}

    epochs = 100

    for i in range(5):
        X_in = Input(shape=(X_train_fft.shape[1],), name="cam_layer")
        x = X_in
        for j in range(i):
            x = Dense(units = 1024, activation="linear")(x)
            x = LeakyReLU(alpha=0.05)(x)
        X_out = Dense(units = 1, activation = 'sigmoid')(x)
        model_i = Model(X_in, X_out)


        best_model_filepath = f"{model_path}/fft_fcn_{i}_layers.h5"
        checkpoint = ModelCheckpoint(best_model_filepath, monitor='val_loss',
                                     verbose=1, save_best_only=True, mode='min')

        model_i.compile(optimizer = Adam(lr=0.0005), loss = 'binary_crossentropy',
                        metrics = ['accuracy'])
        model_i.summary()

        model_i.fit(X_train_fft_sc, y_train, epochs = 100, batch_size = 128,
                       validation_data=(X_test_fft_sc, y_test), callbacks=[checkpoint],
                    class_weight=class_weight)

        from tensorflow.keras.models import load_model

model_loss = []
model_acc = []
model_accs_per_class = []

for i in range(5):
    best_model_filepath = f"{model_path}/fft_fcn_{i}_layers.h5"
    model_i = load_model(best_model_filepath)
    train_acc_ges = model_i.evaluate(X_train_fft_sc, y_train)
    val_acc_ges = model_i.evaluate(X_val_fft_sc, y_val)

    val_acc_1 = model_i.evaluate(X_val_fft_1, y_val_1)
    val_acc_2 = model_i.evaluate(X_val_fft_2, y_val_2)
    val_acc_3 = model_i.evaluate(X_val_fft_3, y_val_3)
    val_acc_4 = model_i.evaluate(X_val_fft_4, y_val_4)
    val_acc_5 = model_i.evaluate(X_val_fft_5, y_val_5)

    model_acc.append([train_acc_ges[1], val_acc_ges[1]])
    model_loss.append([train_acc_ges[0], val_acc_ges[0]])
    model_accs_per_class.append([val_acc_1, val_acc_2, val_acc_3, val_acc_4, val_acc_5])


fig=plt.figure(figsize=(12,8))
ax1=plt.subplot(111)
ax1.plot(np.array(model_loss)[:,1], color="tab:red", marker="o")
ax1.tick_params('y', colors='tab:red')
ax1.set_ylabel(ylabel = "validation loss", color="tab:red")
ax1.set_xticks([0,1,2,3,4])
ax1.set_xlabel("N hidden layers")
ax2 = ax1.twinx()
ax2.plot(np.array(model_acc)[:,1], color="tab:blue", marker="o")
ax2.tick_params('y', colors='tab:blue')
ax2.set_ylabel(ylabel = "validation accuracy", color="tab:blue")
plt.show()

np.array(model_accs_per_class)[0,:,1]

np.array(model_accs_per_class)

from tensorflow.keras.models import load_model
model0 = load_model(f"{model_path}/fft_fcn_0_layers.h5")
model1 = load_model(f"{model_path}/fft_fcn_1_layers.h5")
model2 = load_model(f"{model_path}/fft_fcn_2_layers.h5")
model3 = load_model(f"{model_path}/fft_fcn_3_layers.h5")
model4 = load_model(f"{model_path}/fft_fcn_4_layers.h5")

model2.summary()
accuracies_per_class = []
for i in range(5):
    filepath_i = f"{model_path}/fft_fcn_{i}_layers.h5"
    model_i = load_model(filepath_i)

    val_acc_1 = model_i.evaluate(X_val_fft_1, y_val_1)
    val_acc_2 = model_i.evaluate(X_val_fft_2, y_val_2)
    val_acc_3 = model_i.evaluate(X_val_fft_3, y_val_3)
    val_acc_4 = model_i.evaluate(X_val_fft_4, y_val_4)
    val_acc_5 = model_i.evaluate(X_val_fft_5, y_val_5)
    accuracies_per_class.append([val_acc_1, val_acc_2, val_acc_3, val_acc_4, val_acc_5])

    fig=plt.figure(figsize=(12,8))
ax1=plt.subplot(111, title = "Unbalance Classification Trained with all Unbalances")
unbalances = np.array([0, 4.59e-5, 6.07e-5,7.55e-5,1.521e-4])
ax1.plot(1e6*unbalances, np.array(accuracies_per_class)[0,:,1], label=f"0 hidden layers, mean: {100.0*np.mean(np.array(accuracies_per_class)[0,:,1]):.1f}%", marker="+", ls="--")
ax1.plot(1e6*unbalances, np.array(accuracies_per_class)[1,:,1], label=f"1 hidden layer, mean: {100.0*np.mean(np.array(accuracies_per_class)[1,:,1]):.1f}%", marker="+", ls="--")
ax1.plot(1e6*unbalances, np.array(accuracies_per_class)[2,:,1], label=f"2 hidden layers, mean: {100.0*np.mean(np.array(accuracies_per_class)[2,:,1]):.1f}%", marker="+", ls="--")
ax1.plot(1e6*unbalances, np.array(accuracies_per_class)[3,:,1], label=f"3 hidden layers, mean: {100.0*np.mean(np.array(accuracies_per_class)[3,:,1]):.1f}%", marker="+", ls="--")
ax1.plot(1e6*unbalances, np.array(accuracies_per_class)[4,:,1], label=f"4 hidden layers, mean: {100.0*np.mean(np.array(accuracies_per_class)[4,:,1]):.1f}%", marker="+", ls="--")
plt.ylabel("Accuracy on Evaluation Dataset")
plt.xlabel("Unbalance Factor [mm g]")
plt.legend(bbox_to_anchor=(1.04,1), loc="upper left")
plt.ylim([0.45, 1.05])
plt.grid(True)
plt.tight_layout()
plt.show()

from scipy.stats import mode

def v2rpm(v):
    return 212*v + 209

# 3s ramp up
fade_in = np.arange(0.0, 4.0, 4.0/(3*4096))
# complete voltage sweep
measurement_circle = np.repeat(np.arange(4.0, 8.2, 0.1), 4096*20.0)
# measurement: start-up + 2 voltage sweeps
measurement = np.concatenate([fade_in, np.tile(measurement_circle,3)])
# select the data as actually used
measurement1 = measurement[50000:]
measurement1 = measurement1[:int(len(measurement1)/4096)*4096].reshape(-1,4096)
voltages_measurement = mode(measurement1, axis=1)[0]
voltages_used = np.concatenate([voltages_measurement[:len(X_val_fft_1)],
                                voltages_measurement[:len(X_val_fft_2)],
                                voltages_measurement[:len(X_val_fft_3)],
                                voltages_measurement[:len(X_val_fft_4)],
                                voltages_measurement[:len(X_val_fft_5)]])
rpms_used = v2rpm(voltages_used)

rpm_borders = np.arange(1050, 1975, 25)
errors_per_rpm_range0 = []
errors_per_rpm_range1 = []
errors_per_rpm_range2 = []
errors_per_rpm_range3 = []
errors_per_rpm_range4 = []
for i in range(len(rpm_borders)-1):
    eval_inds = np.where((rpms_used>rpm_borders[i])&(rpms_used<rpm_borders[i+1]))[0]
    errors_per_rpm_range0.append(
        1-np.mean(np.abs(np.int32(model0.predict(X_val_fft_sc[eval_inds])>0.5).reshape(-1)-y_val[eval_inds])))
    errors_per_rpm_range1.append(
        1-np.mean(np.abs(np.int32(model1.predict(X_val_fft_sc[eval_inds])>0.5).reshape(-1)-y_val[eval_inds])))
    errors_per_rpm_range2.append(
        1-np.mean(np.abs(np.int32(model2.predict(X_val_fft_sc[eval_inds])>0.5).reshape(-1)-y_val[eval_inds])))
    errors_per_rpm_range3.append(
        1-np.mean(np.abs(np.int32(model3.predict(X_val_fft_sc[eval_inds])>0.5).reshape(-1)-y_val[eval_inds])))
    errors_per_rpm_range4.append(
        1-np.mean(np.abs(np.int32(model4.predict(X_val_fft_sc[eval_inds])>0.5).reshape(-1)-y_val[eval_inds])))
fig=plt.figure(figsize=(12,8))
plt.plot(np.array(rpm_borders[:-1])+25, errors_per_rpm_range0, marker="+", ls="--", label="0 hidden layers")
plt.plot(np.array(rpm_borders[:-1])+25, errors_per_rpm_range1, marker="+", ls="--", label="1 hidden layer")
plt.plot(np.array(rpm_borders[:-1])+25, errors_per_rpm_range2, marker="+", ls="--", label="2 hidden layers")
plt.plot(np.array(rpm_borders[:-1])+25, errors_per_rpm_range3, marker="+", ls="--", label="3 hidden layers")
plt.plot(np.array(rpm_borders[:-1])+25, errors_per_rpm_range4, marker="+", ls="--", label="4 hidden layers")
plt.ylabel("Accuracy on Evaluation Dataset")
plt.xlabel("Rotation Speed [rpm]")
plt.legend(bbox_to_anchor=(1.04,1), loc="upper left")
plt.ylim([0.45, 1.05])
plt.grid(True)
plt.show()
X_dev = [X0, X1, X2, X3, X4]
y_dev = [y0, y1, y2, y3, y4]

X_val_separated = [X_val_fft_1, X_val_fft_2, X_val_fft_3, X_val_fft_4, X_val_fft_5]
y_val_separated = [y_val_1, y_val_2, y_val_3, y_val_4, y_val_5]

for layer_n in range(5):
    for dataset_i in range(4):
        X_dev_i = np.concatenate([X_dev[0], X_dev[dataset_i+1]])
        y_dev_i = np.concatenate([y_dev[0], y_dev[dataset_i+1]])
        X_val_i = np.concatenate([X_val_separated[0], X_val_separated[dataset_i+1]])
        y_val_i = np.concatenate([y_val_separated[0], y_val_separated[dataset_i+1]])

        train_test_ratio = 0.9
        X_train_i, X_test_i, y_train_i, y_test_i = train_test_split(
            X_dev_i,y_dev_i, test_size = 1-train_test_ratio, random_state = 0)

        X_train_fft_i = np.abs(np.fft.rfft(X_train_i, axis=1))[:,:int(window/2)]
        X_test_fft_i = np.abs(np.fft.rfft(X_test_i, axis=1))[:,:int(window/2)]
        X_train_fft_i[:,0]=0
        X_test_fft_i[:,0]=0

        scaler = RobustScaler(quantile_range=(5,95)).fit(X_train_fft_i)
        X_train_fft_sc_i = scaler.transform(X_train_fft_i)
        X_test_fft_sc_i = scaler.transform(X_test_fft_i)

        if not use_reference_models:
            weight_for_0 = len(y_dev_i)/(2*len(y_dev_i[y_dev_i==0]))
            weight_for_1 = len(y_dev_i)/(2*len(y_dev_i[y_dev_i==1]))
            class_weight = {0: weight_for_0, 1: weight_for_1}

            X_in = Input(shape=(X_train_fft_i.shape[1],), name="cam_layer")
            previous_layer = X_in
            for j in range(layer_n):
                x = Dense(units = 1024, activation="linear")(previous_layer)
                x = LeakyReLU(alpha=0.05)(x)
                previous_layer = x
            X_out = Dense(units = 1, activation = 'sigmoid')(previous_layer)
            model_i = Model(X_in, X_out)

            best_model_filepath = f"{model_path}_fft_fcn_{layer_n}_layers_dataset_pair_{dataset_i}.h5"
            checkpoint = ModelCheckpoint(best_model_filepath, monitor='val_loss',
                                         verbose=1, save_best_only=True, mode='min')
            model_i.compile(optimizer = Adam(lr=0.0005), loss = 'binary_crossentropy',
                            metrics = ['accuracy'])
            model_i.summary()

            model_i.fit(X_train_fft_sc_i, y_train_i, epochs = 100, batch_size = 128,
                       validation_data=(X_test_fft_sc_i, y_test_i), callbacks=[checkpoint],
                        class_weight=class_weight)

        best_model_filepath = f"{model_path}/fft_fcn_{layer_n}_layers_dataset_pair_{dataset_i}.h5"
        model_i = load_model(best_model_filepath)
        train_acc = model_i.evaluate(X_train_fft_sc_i, y_train_i)
        val_acc = model_i.evaluate(X_val_i, y_val_i)
        print(f"Layers: {layer_n}, dataset pair {dataset_i}")
        print(train_acc)
        print(val_acc)

        accuracies = []
for layer_n in range(5):
    accuracies_layer_i = []
    for dataset_i in range(4):
        X_val_i = np.concatenate([X_val_separated[0], X_val_separated[dataset_i+1]])
        y_val_i = np.concatenate([y_val_separated[0], y_val_separated[dataset_i+1]])

        filepath = f"{model_path}/fft_fcn_{layer_n}_layers_dataset_pair_{dataset_i}.h5"
        model_i = load_model(filepath)
        accuracies_layer_i.append(model_i.evaluate(X_val_i, y_val_i)[1])
    accuracies.append(accuracies_layer_i)
accuracies = np.array(accuracies)
accuracies_mean = 100*np.mean(accuracies, axis=1)
fig=plt.figure(figsize=(12,8))
ax1=plt.subplot(111, title = "Pairwise Unbalance Classification")
unbalances = np.array([0, 4.59e-5, 6.07e-5,7.55e-5,1.521e-4])
ax1.plot(1e6*unbalances[1:], accuracies[0,:], label=f"0 hidden  FClayers, mean: {accuracies_mean[0]:.1f}%", marker="+", ls="--")
ax1.plot(1e6*unbalances[1:], accuracies[1,:], label=f"1 hidden  FClayer, mean: {accuracies_mean[1]:.1f}%", marker="+", ls="--")
ax1.plot(1e6*unbalances[1:], accuracies[2,:], label=f"2 hidden  FClayers, mean: {accuracies_mean[2]:.1f}%", marker="+", ls="--")
ax1.plot(1e6*unbalances[1:], accuracies[3,:], label=f"3 hidden  FClayers, mean: {accuracies_mean[3]:.1f}%", marker="+", ls="--")
ax1.plot(1e6*unbalances[1:], accuracies[4,:], label=f"4 hidden  FClayers, mean: {accuracies_mean[4]:.1f}%", marker="+", ls="--")
plt.ylabel("Accuracy on Evaluation Dataset")
plt.xlabel("Unbalance Factor [mm g]")
plt.legend(bbox_to_anchor=(1.04,1), loc="upper left")
plt.ylim([0.45, 1.05])
plt.grid(True)
plt.tight_layout()
plt.show()
fig=plt.figure(figsize=(15,5))
ax1=plt.subplot(132, title = "Unbalance Classification Trained\nwith all Unbalances")
unbalances = np.array([0, 4.59e-5, 6.07e-5,7.55e-5,1.521e-4])
ax1.plot(1e6*unbalances, np.array(accuracies_per_class)[0,:,1],
         label=f"0 hidden FC layers, mean: {100*np.mean(np.array(accuracies_per_class)[0,:,1]):.1f}%", marker="+", ls="--")
ax1.plot(1e6*unbalances, np.array(accuracies_per_class)[1,:,1],
         label=f"1 hidden FC layer, mean: {100*np.mean(np.array(accuracies_per_class)[1,:,1]):.1f}%", marker="+", ls="--")
ax1.plot(1e6*unbalances, np.array(accuracies_per_class)[2,:,1],
         label=f"2 hidden FC layers, mean: {100*np.mean(np.array(accuracies_per_class)[2,:,1]):.1f}%", marker="+", ls="--")
ax1.plot(1e6*unbalances, np.array(accuracies_per_class)[3,:,1],
         label=f"3 hidden FC layers, mean: {100*np.mean(np.array(accuracies_per_class)[3,:,1]):.1f}%", marker="+", ls="--")
ax1.plot(1e6*unbalances, np.array(accuracies_per_class)[4,:,1],
         label=f"4 hidden FC layers, mean: {100*np.mean(np.array(accuracies_per_class)[4,:,1]):.1f}%", marker="+", ls="--")
plt.ylabel("Accuracy on Evaluation Dataset")
plt.xlabel("Unbalance Factor [mm g]")
plt.ylim([0.45, 1.05])
plt.legend(loc="lower right")
ax1.text(-20, 1.05,"(e)", fontsize=12)
ax2=plt.subplot(131, title = "Pairwise Unbalance Classification")
ax2.plot(1e6*unbalances[1:], accuracies[0,:],
         label=f"0 hidden FC layers, mean: {100*np.mean(np.array(accuracies)[0,:]):.1f}%", marker="+", ls="--")
ax2.plot(1e6*unbalances[1:], accuracies[1,:],
         label=f"1 hidden FC layer, mean: {100*np.mean(np.array(accuracies)[1,:]):.1f}%", marker="+", ls="--")
ax2.plot(1e6*unbalances[1:], accuracies[2,:],
         label=f"2 hidden FC layers, mean: {100*np.mean(np.array(accuracies)[2,:]):.1f}%", marker="+", ls="--")
ax2.plot(1e6*unbalances[1:], accuracies[3,:],
         label=f"3 hidden FC layers, mean: {100*np.mean(np.array(accuracies)[3,:]):.1f}%", marker="+", ls="--")
ax2.plot(1e6*unbalances[1:], accuracies[4,:],
         label=f"4 hidden FC layers, mean: {100*np.mean(np.array(accuracies)[4,:]):.1f}%", marker="+", ls="--")
plt.ylabel("Accuracy on Evaluation Dataset")
plt.xlabel("Unbalance Factor [mm g]")
plt.ylim([0.45, 1.05])
plt.legend()
ax2.text(33, 1.05,"(d)", fontsize=12)
title = "Unbalance Classification Trained with all\nUnbalances: Rotation Speed Dependency"
ax3 = plt.subplot(133, title=title)
ax3.plot(np.array(rpm_borders[:-1])+25, errors_per_rpm_range0,
         marker="+", ls="--", label=f"0 hidden FC layers, mean: {100*np.mean(errors_per_rpm_range0):.1f}%")
ax3.plot(np.array(rpm_borders[:-1])+25, errors_per_rpm_range1,
         marker="+", ls="--", label=f"1 hidden FC layer, mean: {100*np.mean(errors_per_rpm_range1):.1f}%")
ax3.plot(np.array(rpm_borders[:-1])+25, errors_per_rpm_range2,
         marker="+", ls="--", label=f"2 hidden FC layers, mean: {100*np.mean(errors_per_rpm_range2):.1f}%")
ax3.plot(np.array(rpm_borders[:-1])+25, errors_per_rpm_range3,
         marker="+", ls="--", label=f"3 hidden FC layers, mean: {100*np.mean(errors_per_rpm_range3):.1f}%")
ax3.plot(np.array(rpm_borders[:-1])+25, errors_per_rpm_range4,
         marker="+", ls="--", label=f"4 hidden FC layers, mean: {100*np.mean(errors_per_rpm_range4):.1f}%")
plt.ylabel("Accuracy on Evaluation Dataset")
plt.xlabel("Rotation Speed [RPM]")
plt.ylim([0.45, 1.05])
plt.legend(loc="lower right")
ax3.text(960, 1.05,"(f)", fontsize=12)
plt.tight_layout()
plt.show()

FileNotFoundError: ignored